In [1]:
from flask import Flask, jsonify, render_template
import psycopg2
from psycopg2.extras import RealDictCursor
import geopandas as gpd
import folium
import pandas as pd
from sqlalchemy import create_engine
from shapely import wkt
from config import db_name, user, password, host

app = Flask(__name__)
app.config['DEBUG'] = True

def get_db_connection():
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=user,
            password=password,
            host=host
        )
        return conn
    except psycopg2.Error as e:
        app.logger.error(f"Error connecting to database: {e}")
        return None


In [4]:
#conn = get_db_connection()

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}/{db_name}')

# SQL query to retrieve the geometries as text
sql = """
SELECT
    cod_reg,
    cod_rip,
    cod_prov,
    ar_kmq,
    nome,
    uid,
    ar_fr_p3p4,
    ar_fr_p2,
    ar_fr_p1,
    pop_fr_p2,
    pop_fr_p1,
    ed_fr_p2,
    ed_fr_p1,
    ar_frp3p4p,
    popfrp3p4p,
    ed_fr_p3p4,
    edfrp3p4p,
    geometry as geom
FROM dataset
"""
#ST_AsText(geometry) as geom
# Step 1: Read the data from the database
df = pd.read_sql(sql, engine)

# Step 2: Convert the text geometries to Shapely geometries
df['geometry'] = df['geom'].apply(wkt.loads)
df = df.drop(columns='geom')  # Drop the text geometry column

# Step 3: Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Step 4: Set the correct CRS (Coordinate Reference System)
gdf.set_crs(epsg=4326, inplace=True)  # Replace 4326 with your actual SRID

# Check the GeoDataFrame

print(gdf.crs)

# Close the SQLAlchemy engine
engine.dispose()
gdf.plot()

GEOSException: ParseException: Unknown type: '01060000000100000001030000000100000079030000B037F842FE7B1D418638D685F7785341502EFFE1E97E1D41B459F505DF7853415050FC9880911D411C0DE0C5EC785341B0627F99A4921D4146B6F3C5EF78534190218E3564971D41C8BAB80558795341D0AACFD5639A1D41DE9387859F795341807346D4679C1D4150AF940592795341500E2D7257A01D415227A0C56D795341B0847C106BA51D41AAF1D285397953415027A0C9A1B21D410E9C33C660785341408B6C27CAB41D41F2D24D863978534130E6AE6567BA1D416ADE7146B9775341B02E6E6307BE1D41948785C68D77534180B74002BFBF1D41C2A8A4C67F775341A0923A01A3C11D41081B9E067877534150499D0093C31D41744694467177534110B7D10053C51D41567DAEC66077534120DBF9FEBEC61D419EEFA74649775341E0361A80FAC71D4132C4B106337753410012147FCECB1D41441CEB86E57653413008AC1CBED11D41703D0A47DC765341B03E57DB0DD41D4164CC5DA72976534150EC2FBBADD41D41C286A7475A75534100098A1F4AC81D41FA7E6A08EF73534130772D2126C71D411E386708F7735341D05D4B88C2C01D41728A8EC82374534170DE710A5FBC1D41006F81883D745341F0B9DA0A07B81D413480B788CD73534180832F0C17B91D

In [3]:
# plot polygons and make a chloropeth map according to an attribute
m = folium.Map(location=[45.46, 9.19], zoom_start=11, tiles='CartoDB positron')

# folium.GeoJson(milan).add_to(m)

folium.Choropleth(
    geo_data= gdf.to_json(),
    data=gdf,
    columns=[gdf.index,'edfrp3p4p'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='edfrp3p4p',
    name='Milano Zones'
).add_to(m)

folium.LayerControl().add_to(m)
m

